In [2]:
import cv2 as cv
import pandas as pd
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import os
import shutil

### Crear una nueva carpeta con las todas las fotos y sus nuevas versionaes, separadas y clasificadas por none, piedra, papel, tijeras

In [14]:
os.mkdir('dataset')
os.mkdir('dataset\\none')
os.mkdir('dataset\\piedra')
os.mkdir('dataset\\papel')
os.mkdir('dataset\\tijeras')

In [13]:
import shutil
shutil.rmtree('dataset')

In [8]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
  return result

In [15]:

for root, dirs, files in os.walk(".", topdown=False):
    # print(root)
    if len(files) != 0:
        clave = ''
        if root.__contains__('none'):
            clave = 'none'
        if root.__contains__('tijeras') or root.__contains__('tijera'):
            clave = 'tijeras'
        if root.__contains__('papel'):
            clave = 'papel'
        if root.__contains__('piedra'):
            clave = 'piedra'
        if clave != '':
            for name in files:
                ruta = os.path.join(root, name)
                img = cv.imread(ruta)
                img = cv.resize(img,(512,512))

                rotated_180 = cv.rotate(img, cv.ROTATE_180)
                rotated_45 = rotate_image(img,45)
                rotated_315 = rotate_image(img,315)

                shutil.copy(ruta, f"dataset/{clave}/{name}")
                cv.imwrite(f"dataset/{clave}/rotated_180_{name}", rotated_180)
                cv.imwrite(f"dataset/{clave}/rotated_45_{name}", rotated_45)
                cv.imwrite(f"dataset/{clave}/rotated_315_{name}", rotated_315)